### Set environment

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse
import json
import os.path
from joblib import dump, load
import google.oauth2.credentials
import pandas_gbq
from itertools import zip_longest

In [2]:
import pygsheets
from get_data import bigquery_data

In [3]:
import time
start=time.perf_counter()

In [4]:
from datetime import datetime  
update_time = datetime.fromtimestamp(time.time())

In [ ]:
bigquery_data.get_bigquery_credential()

### Get Vendor Name

In [ ]:
df = bigquery_data.get_sql_data("sql/fp_vendors.sql")
df

### Google Places SearchText API

In [ ]:
vendor_data_dict = df['vendor_name'].to_dict()
print(vendor_data_dict)

In [8]:
responses = []
vendor_name = []

In [ ]:
f = open('google_key.json') # add Google key
google_key = json.load(f)

In [ ]:
import requests
from itertools import islice

url = 'https://places.googleapis.com/v1/places:searchText'

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': google_key['X-Goog-Api-Key_4'],
    'X-Goog-FieldMask': 'places.displayName,places.businessStatus,places.rating,places.userRatingCount,places.priceLevel'
}

chunk_size = 10
vendor_data_values = list(vendor_data_dict.values())

for chunk_start in range(0, len(vendor_data_values), chunk_size):
    chunk = vendor_data_values[chunk_start:chunk_start + chunk_size]
    
    for value in chunk:
        data = {
            "textQuery": value,
            "languageCode": "zh-TW",
            "maxResultCount": 1,
            "locationRestriction": {
                "rectangle": {
                    "low": {
                        "latitude": 21,
                        "longitude": 119
                    },
                    "high": {
                        "latitude": 26,
                        "longitude": 123
                    }
                }
            }
        }
        response = requests.post(url, headers=headers, json=data)
        responses.append(response.json())
        vendor_name.append(value)
        
# print("responses:", responses)
# print("vendor name:", vendor_name)

In [12]:
ratings = []
business_statuses = []
price_levels = []
user_rating_counts = []
display_names = []

In [13]:
for response in responses:
    if response:
        place = response['places'][0]
        ratings.append(place.get('rating', 0))
        business_statuses.append(place.get('businessStatus', "NA"))
        price_levels.append(place.get('priceLevel', "NA"))
        user_rating_counts.append(place.get('userRatingCount', 0))
        display_name = place.get('displayName', {}).get('text', "NA")
        display_names.append(display_name)
    else:
        ratings.append(0)
        business_statuses.append("NA")
        price_levels.append("NA")
        user_rating_counts.append(0)
        display_names.append("NA")

In [14]:
source = len(vendor_name) * ["fp"]

In [15]:
result = pd.DataFrame({'source': source,'vendor_name':vendor_name, 'google_display_names': display_names,'google_ratings': ratings, 'google_reviews': user_rating_counts, 'google_business_status':business_statuses, 'google_price_levels': price_levels})
result

,source,vendor_name,google_display_names,google_ratings,google_reviews,google_business_status,google_price_levels
0,fp,鐵板饄 (基隆海大店),基隆醬鐵板燒,3.7,182,OPERATIONAL,PRICE_LEVEL_INEXPENSIVE
1,fp,微風甜甜食品有限公司,微甜食品有限公司,3.5,2,OPERATIONAL,NA
2,fp,成月越南小吃,成月越南小吃店,5.0,2,OPERATIONAL,NA
3,fp,好了啦紅茶冰 (林園仁愛店),好了啦紅茶冰－林園仁愛店,5.0,5,OPERATIONAL,PRICE_LEVEL_INEXPENSIVE
4,fp,麥味登 (嘉義友愛店),嘉義麥味登友愛店,4.3,33,OPERATIONAL,NA
...,...,...,...,...,...,...,...
445,fp,肥嘟嘟燒肉飯,胖嘟嘟燒肉飯,4.3,101,OPERATIONAL,NA
446,fp,蠔串 (林口旗艦店),蠔串-火鍋串烤店,4.6,2520,OPERATIONAL,PRICE_LEVEL_MODERATE
447,fp,辣比小辛麻辣干鍋 (台中豐原店),小辛干麻辣干鍋｜西屯總店｜客製化干鍋｜台中美食｜中式料理｜巷弄美食｜晚餐宵夜首選,4.9,128,OPERATIONAL,NA
448,fp,林記德國豬腳,林家德國豬腳,4.8,176,OPERATIONAL,PRICE_LEVEL_INEXPENSIVE


### Upload dataframe to GCP

In [22]:
import pandas as pd
from datetime import datetime
import re
import google.oauth2.credentials
import pandas_gbq
import json

In [ ]:
f = open('user_account.json') # add GCP token
credential = json.load(f)

credentials = google.oauth2.credentials.Credentials(
    credential['token'],
    refresh_token=credential['refresh_token'],
    token_uri=credential['token_uri'],
    client_id=credential['client_id'],
    client_secret=credential['client_secret']
    )

pandas_gbq.context.credentials = credentials

In [ ]:
project_id = "database_name"
table_id = 'table_name'

result.to_gbq(destination_table = table_id, 
              project_id = project_id, 
              credentials = credentials, 
              if_exists='append',
              table_schema=table_schema)

### Complete Google API

In [26]:
end=time.perf_counter()

In [ ]:
print("執行時間: %f 秒" % (end-start))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=63d13dd8-1e3d-4d6d-8c14-b5ff0c503a9a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>